In [ ]:
import time, sys

def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        sys.stdout.write('\r' + str(timeformat))
        sys.stdout.flush()
        time.sleep(1)
        t -= 1
    print("Time's Up! \n")

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('max_rows', 15)
pd.set_option('max_columns', 10)
pd.set_option('notebook_repr_html', True)

import numpy as np
%pylab inline

In [ ]:
print pd.__version__

### 1 - Import Data with `read_csv()`

In [ ]:
pd.read_csv?

In [ ]:
# Import the Data
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python_reference/master/Data/some_soccer_data.csv')
df.head()

In [ ]:
type(df)

#### Check the dimensions of the imported data with `.shape`

In [ ]:
df.shape

** Check the data types for each column **

In [ ]:
df.dtypes

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

---
### 2 - Edit column names with `.rename()`

In [ ]:
# Edit column names
df.columns = [x.lower() for x in df.columns]
df.columns.values

In [ ]:
# Rename specific columns
df = df.rename(columns={'gp': 'gamesPlayed',
               'g': 'goals',
               'a': 'assists',
               'sot': 'shotsOnTarget',
               'ppg': 'pointsPerGame',
               'p': 'points'})
df.columns

---
### 3 - Change values in a column with `.apply()`

In [ ]:
df.salary.head()

In [ ]:
df.salary.map(lambda x: x.strip('$m')).head()

In [ ]:
# Change entries in a column by row
df.salary = (df
             .loc[:, 'salary']
             .map(lambda x: x.strip('$m'))
             .astype(float))

In [ ]:
df

> Note:

`.apply()` on a single column has the same effect as using `map()`

---
### 4 - Adding Columns to a DF

In [ ]:
df.player[1]

In [ ]:
df.player[1].split('\n')

In [ ]:
# Derive new columns from the 'player column'

In [ ]:
def processPlayerCol(text):
    """
    This function extracts name, position and club from the player column
    """
    name, rest = text.split('\n')
    position, club = [x.strip() for x in rest.split('—')]
    return Series([name, position, club], index=['player', 'position', 'club'])

In [ ]:
processPlayerCol(df.player[9])

In [ ]:
df.join(DataFrame(df['player'].map(lambda x: processPlayerCol(x)).tolist()), lsuffix='_l').drop('player_l', axis=1)

### Time Comparison for Type Conversion (single values, series) functions

In [ ]:
int('12345')

In [ ]:
float('12345')

In [ ]:
%timeit Series(['123', '456', '789']).astype(int)

In [ ]:
%timeit Series(['123', '456', '789']).map(int)

In [ ]:
[int(x) for x in ['123', '456', '789']]

In [ ]:
%timeit Series([int(x) for x in ['123', '456', '789']])

> Using List Comprehensions is FASTER for element-wise transformations

---
### 5 - Dropping columns from a DF with `.drop(labels, axis)`

In [ ]:
df

In [ ]:
df['player2'] = df.player.map(lambda x: x.split('\n')[0])
df['club'] = df.player.map(lambda x: x.split('—')[1])
df['position'] = df.player.map(lambda x: x.split('—')[0].split('\n')[1])

In [ ]:
df[:3]

In [ ]:
df.drop('player', axis=1, inplace=True)

In [ ]:
# df = df.drop(labels='player', axis=1)
df.head()    

In [ ]:
df.rename(columns={'player2':'player'}, inplace=True)

In [ ]:
df.head()

---
### 6 - Applying a function to multiple columns with `applymap()`

In [ ]:
# Change the created columns to lowercase
cols = ['player', 'position', 'club']
df[cols].head()

In [ ]:
df[cols] = df[cols].applymap(lambda x: x.lower())
df[cols].head()

In [ ]:
num_cols = df.describe().columns.tolist()

In [ ]:
print df[num_cols].astype(float)
print df[num_cols].applymap(float)
print df[num_cols].applymap(lambda x: float(x))

---
### 7 - Deal with missing data using `isnull()`, `dropna()` and `fillna()`

#### Remove rows with missing data using `.dropna()`

In [ ]:
df

In [ ]:
df.dropna()

#### Test % of rows with missing data

In [ ]:
print('%d rows have missing values' % (df.shape[0] - df.dropna().shape[0]))

#### Select rows with missing/non-missing data in a specific column 

In [ ]:
# On which players do we not have data for assists?
df[df.assists.isnull()]
# also see -- .notnull()

In [ ]:
df[df.assists.notnull()]

---
### Impute missing data `.fillna()`

In [ ]:
# Impute with Median
df['assists'].fillna(int(df.assists.median()), inplace=True)

# Impute with Mean
df['gamesPlayed'].fillna(int(df.gamesPlayed.mean()), inplace=True)

df[['player', 'assists', 'gamesPlayed']]

---
### 8 - Sorting using `.sort()` or `sort_index()`

In [ ]:
df.columns.values

In [ ]:
# Find the top 3 goalscorers
df.sort_values(by=['goals'], ascending=False, inplace=True)
df[['player', 'goals']].head(3)

In [ ]:
df

In [ ]:
# Drop old index, create new
df.reset_index().drop('index', axis=1)

In [ ]:
(df
 .reset_index()
 .drop('index', axis=1)
 .set_index('player'))

### 9 - Modifying the index `set_index()` and `reindex()`

In [ ]:
df.set_index('player', inplace=True)

In [ ]:
df[:3]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df[:3]

---
### 10 - Subsetting data using Booleans

In [ ]:
df['club'] = df.club.map(lambda x: x.strip())
df['position'] = df.position.map(lambda x: x.strip())

In [ ]:
%timeit df[(df['club'] == 'arsenal') | (df['club'] == 'chelsea') ]

In [ ]:
# Alternate way of selecting only Arsenal and Chelsea players
%timeit df[df.club.isin(['arsenal', 'chelsea'])]

In [ ]:
%timeit df[[x in ['arsenal', 'chelsea'] for x in df.club]]

In [ ]:
%timeit df.query("club in ('arsenal', 'chelsea')")

In [ ]:
# Selecting forwards from Arsenal only
df[ (df['club'] == 'manchester city') & (df['position'] == 'forward') ]

### 11 - GroupBy operations

- `agg, apply`

In [ ]:
# For each position, find mean and sum of goals and assists

print (df
 .groupby('position')[['goals', 'assists']]
 .agg(['sum', 'mean'])
 .stack()
)